In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
path = '../input/leaf-classification/train.csv.zip'
df = pd.read_csv(path,index_col='id')
df

In [ ]:
print ('  Data Types        ')
for i in df.columns:
    print(df[i].dtypes)
print ('----------------------------------')
print ('   Counts of Missing values')
print ('----------------------------------')

for i in df.columns:
    print(df[i].isna().sum())


In [ ]:
y = df['species']
X = df.drop('species', axis=1)

# Split Data

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.9,random_state=0)

# XGBoost

In [ ]:
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, log_loss

In [ ]:
XGB_model = XGBClassifier(n_estimators=100, learning_rate=0.5)

In [ ]:
XGB_model.fit(X_train, y_train)

In [ ]:
predictions = XGB_model.predict(X_valid)
print("accuracy_score: " + str(accuracy_score(predictions, y_valid)))

In [ ]:
my_list = []
for k in y:
    my_list.append(k)
    
len (my_list)

In [ ]:
classes = set (my_list)
classes
len (classes)

In [ ]:
X

In [ ]:
from sklearn.model_selection import GridSearchCV

parameters = [{
    'n_estimators': list(range(100, 201, 100)), 
    'learning_rate': [0.2,0.5,0.7,0.9], 
    'max_depth': list(range(6, 40, 10))
}]
gsearch = GridSearchCV(estimator=XGBClassifier(),
                       param_grid = parameters, 
                       scoring= 'neg_log_loss',
                       n_jobs=4,cv=5, verbose=7)


In [ ]:
gsearch


In [ ]:
final_model = XGBClassifier(n_estimators=100, 
                          learning_rate=0.5, 
                          max_depth=6)
final_model.fit(X,y)


In [ ]:
test = pd.read_csv('../input/leaf-classification/test.csv.zip')
test_index = test.id 
test.drop('id',axis=1,inplace=True)
pred_test = final_model.predict_proba(test)
pred_test


In [ ]:
submission = pd.DataFrame(pred_test, columns=classes)
submission.insert(0, 'id', test_index)
submission.to_csv('submission_final.csv', index=False)
print('DONE!')

In [ ]:
submission